###### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
 

In [1]:
#import libraries
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Input, BatchNormalization, AveragePooling2D, Dense, Conv2D , Flatten ,DepthwiseConv2D, ReLU ,GlobalAveragePooling2D , Dropout
import keras_tuner
from tensorflow import keras

# # Preparing Dataset

In [2]:
photo_size = 224
batch_size=32
def prepare_dataset(data_dir):
    datagen = ImageDataGenerator(
        rescale=1 / 255,
        rotation_range=40,
        width_shift_range=.2,
        height_shift_range=.2,
        shear_range=.1,
        horizontal_flip=True,
        fill_mode='nearest',
        zoom_range=.2,
    )
    generator = datagen.flow_from_directory(
        data_dir,
        target_size=(photo_size,photo_size),
        class_mode='binary',
        batch_size=batch_size,
        classes=['non_autistic','autistic']
    )
    return generator

# Load Dataset

In [3]:
#load dataset
traindata = prepare_dataset("/kaggle/input/autistic-children-facial-data-set/train")
testdata= prepare_dataset("/kaggle/input/autistic-children-facial-data-set/test")
validData = prepare_dataset("/kaggle/input/autistic-children-facial-data-set/valid")

Found 2536 images belonging to 2 classes.
Found 300 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


# Improved MobileNet v1

In [4]:
def depth_block(x, strides):
    x = DepthwiseConv2D(3,strides=strides,padding='same',  use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x
def single_conv_block(x,filters):
    x = Conv2D(filters, 1,use_bias=False)(x)
    x= BatchNormalization()(x)
    x = ReLU()(x)
    return x
def combo_layer(x,filters,strides):
    x = depth_block(x,strides)
    x = single_conv_block(x, filters)
    return x

    

In [5]:
def build_model(hp):
    n_classes = 1
    input_shape=(224,224,3)
    input =Input(input_shape)
    x = Conv2D(32,3,strides=(2,2),padding = 'same', use_bias=False) (input)
    x =  BatchNormalization()(x)
    x = ReLU()(x)
    x = combo_layer(x,64, strides=(1,1))
    x = combo_layer(x,128,strides=(2,2))
    x = combo_layer(x,128,strides=(1,1))
    x = combo_layer(x,256,strides=(2,2))
    x = combo_layer(x,256,strides=(1,1))
    x = combo_layer(x,512,strides=(2,2))
    for _ in range(5):
        x = combo_layer(x,512,strides=(1,1))
    x = combo_layer(x,1024,strides=(2,2))
    x = combo_layer(x,1024,strides=(1,1))

    x = GlobalAveragePooling2D()(x)
    output = Dense(n_classes,activation='sigmoid')(x)

    model = Model(input, output)

    model.compile( optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3,1e-4])),loss=tf.keras.losses.BinaryCrossentropy(),metrics=['acc'])

    return model

In [6]:
tuner =  keras_tuner.RandomSearch(
                       build_model,
                       objective = "val_acc", #optimize val acc
                       max_trials=5, 
                       executions_per_trial=3,
                       directory='/kaggle/working/', #Saving dir
                       project_name='mobilenetv1_hpTunning')
tuner.search_space_summary()

2023-02-21 17:26:16.192364: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 17:26:16.334066: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 17:26:16.334906: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 17:26:16.336899: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Search space summary
Default search space size: 1
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


# Fit and Compile Model

In [7]:
def cal_steps(num_images, batch_size):
    # calculates steps for generator
    steps = num_images // batch_size
    # adds 1 to the generator steps if the steps multiplied by
    # the batch size is less than the total training samples
    return  steps

In [8]:
checkpoint = ModelCheckpoint("/kaggle/working/mobilenet_v1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False)
early = EarlyStopping(monitor='val_acc',patience=30, verbose=1, mode='max')
tuner.search(traindata, validation_data= validData, validation_steps=cal_steps(validData.samples, batch_size),epochs=50 ,steps_per_epoch=cal_steps(traindata.samples, batch_size))

Trial 3 Complete [01h 36m 19s]
val_acc: 0.8194444378217062

Best val_acc So Far: 0.8333333333333334
Total elapsed time: 04h 50m 12s


In [9]:
tuner.results_summary()

Results summary
Results in /kaggle/working/mobilenetv1_hpTunning
Showing 10 best trials
Trial summary
Hyperparameters:
learning_rate: 0.0001
Score: 0.8333333333333334
Trial summary
Hyperparameters:
learning_rate: 0.001
Score: 0.8194444378217062
Trial summary
Hyperparameters:
learning_rate: 0.01
Score: 0.8020833333333334


In [10]:
best_hp = tuner.get_best_hyperparameters(1)[0]
print(best_hp.values)

{'learning_rate': 0.0001}


In [11]:
#checkpoint = ModelCheckpoint("/kaggle/working/mobilenet_v1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False)
#early = EarlyStopping(monitor='val_acc',patience=30, verbose=1, mode='max')
#hist = model.fit(traindata, validation_data= validData, validation_steps=cal_steps(validData.samples, batch_size),epochs=100 ,steps_per_epoch=cal_steps(traindata.samples, batch_size),callbacks=[checkpoint,early])

# Plotting History of Model

In [12]:
#import matplotlib.pyplot as plt
#plt.plot(hist.history["acc"])
#plt.plot(hist.history['val_acc'])
#plt.plot(hist.history['loss'])
#plt.plot(hist.history['val_loss'])
#plt.title("model accuracy")
#plt.ylabel("Accuracy")
#plt.xlabel("Epoch")
#plt.legend(["Accuracy","Validation Accuracy"])
#plt.show()

In [13]:
#model.evaluate(traindata)

In [14]:
#model.evaluate(validData)

In [15]:
#model.evaluate(testdata)

In [16]:
# print(cal_steps(traindata.samples, batch_size))